In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.0

# 4 함수형 프로그래밍 구조 알아보기

## 4-1 고차함수

## 4-1-1 고차함수 처리 알아보기 

In [2]:
typealias f = (Int,Int)-> Int                     // 함수 자료형을 타입별칭으로 지정

In [3]:
fun highfunc(vararg x:Int, op:f) : Int {          // 함수를 매개변수로 받는 고차함수                     
    return x.toList().reduce(op)                  // 배열을 리스트로 변환하고 리듀스로 전달된 함수를 실행 
}

println(highfunc(1,2,3,4,op={x:Int,y:Int->x+y}))  // 합산을 하는 람다표현식 전달 

10


In [4]:
fun add(x:Int, y:Int) : Int = x+y
println(highfunc(1,2,3,4,5, op=::add))            // 합산을 하는 함수 참조 전달 


fun highOrder() : f {                              // 함수를 반환하는 고차 함수 
    return {x,y -> x+y}                            // 합산을 하는 람다표현식 반환 
}

println(highOrder()(10,20))                        // 함수를 실행

15
30


## 4-1-2 고차함수 일반화하기 

In [5]:
typealias g = (Int,Int)-> Int                     // 함수 자료형을 타입별칭으로 지정

fun agg(nums:IntArray, op: g ): Int {             // 함수를 정의한다. 
    var result = nums.firstOrNull() ?: 0          // 정수 배열의 합산을 순환문으로 처리한다. 
    for (i in 1..nums.lastIndex) {
        result = op(result,nums[i])               // 실제 처리되는 결과는 전달되는 함수의 기능에 따른다. 
    }
    return result
}

In [6]:
println(agg(intArrayOf(1,2,3,4), {x,y -> x+y}))   // 람다표현식을 전달받아 배열을 합산한다. 

10


In [7]:
fun sum(nums:IntArray) :  Int {                   // 함산 함수를 정의한다. 
    return agg(nums, {x:Int,y:Int -> x+y})        // 위에 정의된 고차함수를 함수 내부에서 처리한다.
}

fun max(nums:IntArray) :  Int {                    // 최대값을 찾는 함수를 정의한다
    return agg(nums, 
         {x:Int,y:Int -> if (y > x) y else x })    // 위에서 정의된 고차함수에 최대값을 구한는 람다표현식 전달 
}

println(sum(intArrayOf(10,20,30,40)))              // 합산 처리
println(max(intArrayOf(10,20,30,40)))              // 최대값 처리

100
40


## 4-2 합성함수

## 4-2-1 일반 함수를 사용한 합성함수

In [8]:
fun composeF(f: (Int) -> Int, g:(Int) -> Int):         // 동일한 매개변수 갯수를 받는 두 개의 함수를 매개변수로 받는다. 
                                    (Int) -> Int {     // 반환값은 두 함수를 연결한 하나의 함수 
    return { p1: Int -> f(g(p1)) }                     // 두 함수를 하나로 합성한 함수를 반환한다. 
}
                                    
val f = {x : Int -> x+2}                               // 첫번째 함수
val g = {y : Int -> y+3}                               // 두번째 함수 : 함수 내부에 결합되는 함수
val composeFunc = composeF(f, g)                       // 두 개의 함수를 인자로 전달한다.

println(f(g(3)))                                       // 두 함수를 결합해서 실행하기 
println(composeFunc(3))                                // 합성함수로 반환된 함수 실행하기 


fun composeR(g: (Int) -> Int, f: (Int) -> Int):        // 역방향으로 합성함수를 만든다 
                                     (Int) -> Int {
    return { p1: Int -> g(f(p1)) }                     // 역방향으로 실행된다 
}

val g1 = {y : Int -> y + 3}                            // 첫반째 함수 정의 
val f1 = {x : Int -> x + 2}                            // 두번째 함수 정의
val composeFuncR = composeR(g1,f1)

println(g1(f1(3)))                                     // 역방향으로 함수를 실행
println(composeFuncR(3))                               // 역방향으로 합성된 함수를 실행

8
8
8
8


## 4-2-2  확장함수의 합성함수

In [9]:
typealias F  = (Int) -> Int                        // 별칭으로 타입지정 
typealias G  = (Int) -> Int
typealias FG = (Int) -> Int
typealias GF = (Int) -> Int

infix fun F.compose(g: G ) :FG {                   // 첫번째 함수의 확장함수를 지정 
    return { p1: Int -> this(g(p1)) }              // 입력받은 함수를 첫번째 함수내수에서 실행한고 
}                                                  // 첫번째 함수도 실행 

infix fun F.then(g: G): GF {                       // 첫번째 함수의 인자로 다른 함수의 인자로 받음
    return { p1: Int -> g(this(p1)) }              // 두번째 함수에 첫번째 함수 실행된 결과를 전달 
}

val plus2: F  = { it + 2 }                         // 첫번째 함수 정의 
val times3: G = { it * 3 }                         // 두번째 함수 정의

val plus2times3 = plus2 compose times3             // 인픽스로 지정해서 점연산자 없이 사용 
                                                   // 정방향으로 결합 
println(plus2times3(3))                                
println(plus2(times3(3)))
println(plus2times3(3) == 11)

val times3plus2 =  plus2 then times3               // 역방향으로 함수 결합
println(times3plus2 (4))
println(times3(plus2(4)))
println(times3plus2(4) == 18)

11
11
true
18
18
true


## 4-3 함수체인

## 4-3-1  함수체인

In [10]:
fun Outer(x:Int) : (Int) -> Int {              // 외부함수를 정의
    fun inner1(y:Int) : Int {                  // 내부함수를 정의 
        return x+y
    }
    return ::inner1                            // 내부함수를 반환 
}

println(Outer(100)(200))                       // 함수를 연속실행 
val out = Outer(100)
println(out(200))
                                              // 람다표현식으로 함수의 내부 계층을 만듬
val lambda = { x :Int -> { y:Int -> {z :Int -> x+y+z}} }

println(lambda(100)(200)(300))                // 함수를 연속으로 실행 

300
300
600


## 4-3-2 메서드 체인 

In [11]:
class Car(var ownerName: String, var color: String) {       // 클래스 정의 
    fun changeOwner(newName: String) :Car {                 // 메소드 정의 
        this.ownerName = newName
        return this                                         // 연속 호출을 위해 객체 반환 
    }

    fun repaint(newColor: String) :Car {                    // 메소드 정의                   
        this.color = newColor
        return this                                         // 연속 호출을 위한 객체 반환
    }
    
    fun info() : Unit =                                     // 메소드 정의 
          println("Car(소유자 = $ownerName, 색상= $color)")    // 최종 처리 결과 
}

val c = Car("서정희", "빨간색")                                 // 객체 생성 
c.info()
c.changeOwner("이재헌").repaint("파란색").info()                // 메소드 체인 처리

Car(소유자 = 서정희, 색상= 빨간색)
Car(소유자 = 이재헌, 색상= 파란색)


## 4-3-3 확장함수를 사용한 체인

In [12]:
data class Car1(var ownerName: String, var color: String)       // 데이터클래스 정의 

fun Car1.changeOwner(newName: String) :Car1 {                   // 확장함수 정의 
        this.ownerName = newName
        return this                                             // 리시버 객체 반환
}

fun Car1.repaint(newColor: String) :Car1 {                      // 확장함수 정의 
        this.color = newColor
        return this                                             // 리시버 객체 반환
}
    
fun Car1.info() : Unit =                                        // 확장 함수 
    println("Car(소유자 = $ownerName, 색상= $color)")              // 메소드 체인 종료 

val cc = Car1("우미선", "하얀색")                                   // 객체생성
cc.info()
cc.changeOwner("좌미선").repaint("노란색").info()                   // 메소드 체인 처리

Car(소유자 = 우미선, 색상= 하얀색)
Car(소유자 = 좌미선, 색상= 노란색)
